##**The Neural Translation Model (NMT)**

For the NMT the network (a system of connected layers/models) used for training differs slightly from the network used for inference. Both use the the seq-to-seq encoder-decoder architecture. 




###**The Inference Mode**

**Encoder**

The inference time encoding follows the same steps as training time encoding.

<br>

**Decoder (No attention)**

During training time, we passed a `batch_size(num_sents) x max_sentence_length` array representing the target words into the decoder lstm. The decoder_lstm learns how to represent a given target sentence using the context from the encoder lstm (that learns to represent a source sentence).  

At test time, several things are different:

1. We no longer have access to a complete translation of the source sentence (recall that no target_words array exists for dev and test sets). Rather, we initialize the target_words_array as thus:

    Each expected sentence contains only a single token index, the index of the `'<start>'` token. So, the target_word_dev/test is a `batch_size x 1` array. (see the nmt.eval() function for this)

2. This `batch_size x 1` array is fed to the trained decoder_lstm and the predicted array is a `batch_size x 1 x target_vocab_size` such that taking the argmax of this array accross the dimension 2 will give the most probable next word. 

    For example, at time_step `0`, the first time step, where the `step_target_words` given is the `batch_size x 1` array containing the `'<start>'` token, the next word prediction of the decoder is for each sentence (in the batch) the initial word in the sentence. 

3. At the first time step, the decoder_lstm still uses the encoder_states as it's initial states. At subsequent time steps, it uses it's own states from the previous time steps. This is also what the decoder_lstm does at training time but it is made more explicit here as we loop over time steps using a for loop.
(see nmt.eval())





##**Training Without Attention**

In [ ]:
! pip install keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# change this to the path to your folder. Remember to start from the home directory
PATH = 'MyDrive/nmt_lab_files' 

In [ ]:
PATH_TO_FOLDER = "/content/drive/" 

In [ ]:
import sys
sys.path.append(PATH_TO_FOLDER)

In [ ]:
import nmt_model_keras
import importlib
importlib.reload(nmt_model_keras)

<module 'nmt_model_keras' from '/content/nmt_model_keras.py'>

In [ ]:
SOURCE_PATH = '/content/data.30.vi' 
TARGET_PATH = '/content/data.30.en' 

In [ ]:
import nmt_model_keras as nmt 

In [ ]:
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=False)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder
						 Train Model Summary.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
source_embed_layer (Embedding)  (None, None, 100)    203400      input_1[0][0]                    
______

##**Decoding with Attention**

The inputs to the attention layer are encoder and decoder outputs. The attention mechanism:
1. Computes a score (a luong score) for each source word
2. Weights the words by their luong scores.
3. Concatenates the wieghted encoder representation with the decoder_ouput.
This new decoder output will now be the input to the decoder_dense layer. 

Task 3 description in the doc file outlines the steps for this in detail. Once you have completed this Task, you are now ready to train with attention. Training time will be no more than 10 minutes using a GPU and you should get a bleu score of about 15.

In [ ]:
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=True)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder
						 Train Model Summary.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
source_embed_layer (Embedding)  (None, None, 100)    203400      input_6[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None)]       0                                            
____

In [2]:
from prettytable import PrettyTable



In [8]:
my_table  = PrettyTable([" Model_Type", "Blue_Score","Epochs"])

In [9]:
my_table.add_row(["NMT without Attention","4.8","10"])
my_table.add_row(["NMT with Attention","15.6","10"])

In [10]:
print(my_table)

+-----------------------+------------+--------+
|       Model_Type      | Blue_Score | Epochs |
+-----------------------+------------+--------+
| NMT without Attention |    4.8     |   10   |
|   NMT with Attention  |    15.6    |   10   |
+-----------------------+------------+--------+
